In [ ]:
%run block_swipdg.ipynb

In [ ]:
import numpy as np

from dune.xt.grid import make_walker_on_dd_subdomain_part as make_subdomain_walker

from dune.gdt import (
    make_rt_leaf_view_to_2x1_pdelab_p0_space
        as make_rt_space,
    RS2017_apply_diffusive_flux_reconstruction_in_neighborhood
        as apply_diffusive_flux_reconstruction_in_neighborhood,
    apply_oswald_interpolation_operator
        as apply_oswald_interpolation_operator_on_neighborhood,
    make_l2_localizable_product_on_dd_subdomain_part_for_1x1_range_times_1x1_source
        as make_localizable_subdomain_l2_product,
    RS2017_make_elliptic_matrix_operator_on_subdomain
        as make_elliptic_matrix_operator_on_subdomain,
    RS2017_make_Hdiv_semi_product_matrix_operator_on_subdomain
        as make_Hdiv_semi_product_matrix_operator_on_subdomain,
    RS2017_make_residual_part_vector_functional_on_subdomain
        as make_residual_part_vector_functional_on_subdomain,
    RS2017_residual_indicator_min_diffusion_eigenvalue
        as min_diffusion_eigenvalue,
    RS2017_residual_indicator_subdomain_diameter
        as subdomain_diameter,
    RS2017_make_diffusive_flux_aa_product_matrix_operator_on_subdomain
        as make_diffusive_flux_aa_product,
    RS2017_make_diffusive_flux_ab_product_matrix_operator_on_subdomain
        as make_diffusive_flux_ab_product,
    RS2017_make_diffusive_flux_bb_product_matrix_operator_on_subdomain
        as make_diffusive_flux_bb_product,
    RS2017_make_penalty_product_matrix_operator_on_oversampled_subdomain
        as make_penalty_product_matrix_operator
)

print('estimating error ', end='')

# fake parametric situation
lambda_bar = lambda_
lambda_hat = lambda_
lambda_xi = lambda_
lambda_xi_prime = lambda_
alpha_mu_mu_bar = 1.
gamma_mu_mu_bar = 1.
alpha_mu_mu_hat = 1.

subdomain_spaces = [block_space.local_space(ii) for ii in range(block_space.num_blocks)]
# localized_u_h contains the vector of the solution restricted to each subdomain
local_sizes = [subdomain_spaces[ii].size()
               for ii in range(block_space.num_blocks)]
local_starts = [int(np.sum(local_sizes[:ii]))
                for ii in range(block_space.num_blocks)]
local_starts.append(block_space.mapper.size)
localized_u_h_np = np.array(u_h_vector, copy=False)
localized_u_h_np = [localized_u_h_np[local_starts[ii]:local_starts[ii+1]]
                    for ii in range(block_space.num_blocks)]
localized_u_h = [Vector(subdomain_spaces[ii].size(), 0.)
                 for ii in range(block_space.num_blocks)]
for ii in range(block_space.num_blocks):
    tmp = np.array(localized_u_h[ii], copy=False)
    tmp[:] = localized_u_h_np[ii][:]

local_boundary_info = make_subdomain_boundary_info(
    grid,
    {'type': 'xt.grid.boundaryinfo.boundarysegmentindexbased',
     'default': 'dirichlet',
     'neumann': '[{} {}]'.format(inner_boundary_id, inner_boundary_id+1)})

global_rt_space = make_rt_space(grid)
subdomain_rt_spaces = [global_rt_space.restrict_to_dd_subdomain_part(grid, ii)
                       for ii in range(grid.num_subdomains)]
neighborhood_rt_spaces = [global_rt_space.restrict_to_dd_subdomain_oversampled_part(grid, ii)
                          for ii in range(grid.num_subdomains)]

local_eta_nc_squared = [0. for ii in range(grid.num_subdomains)]
local_eta_r_squared = [0. for ii in range(grid.num_subdomains)]
local_eta_df_squared = [0. for ii in range(grid.num_subdomains)]
penalty_products = [0. for ii in range(grid.num_subdomains)]

for ii in range(grid.num_subdomains):
    neighborhood = grid.neighborhood_of(ii)
    neighborhood_space = block_space.restricted_to_neighborhood(neighborhood)
    
    subdomain_walker = make_subdomain_walker(grid, ii)
    
    # prepare some operators/products/functionals for assembly/application in one grid walk
    
    # penalty product
    penalty_product = make_penalty_product_matrix_operator(grid,
                                                           ii,
                                                           local_boundary_info,
                                                           neighborhood_space,
                                                           lambda_bar,
                                                           kappa,
                                                           over_integrate=0)
    subdomain_walker.append(penalty_product)
    
    # elliptic product
    elliptic_product = make_elliptic_matrix_operator_on_subdomain(
        grid, ii,
        subdomain_spaces[ii],
        lambda_hat,
        kappa,
        over_integrate=0)
    subdomain_walker.append(elliptic_product)

    # H-dvi semi product
    h_div_semi_product = make_Hdiv_semi_product_matrix_operator_on_subdomain(
        grid, ii,
        subdomain_rt_spaces[ii])
    subdomain_walker.append(h_div_semi_product)

    # eta r, f x f part
    eta_r_fxf_product = make_localizable_subdomain_l2_product(grid, ii, f, f, over_integrate=0)
    subdomain_walker.append(eta_r_fxf_product)

    # eta r, f x reconstructed_u part
    eta_r_fxRu_functional = make_residual_part_vector_functional_on_subdomain(grid, ii,
                                                                              subdomain_rt_spaces[ii],
                                                                              f,
                                                                              over_integrate=0)
    subdomain_walker.append(eta_r_fxRu_functional)

    # eta df, u x u part
    diffusive_flux_aa_product = make_diffusive_flux_aa_product(
        grid, ii,
        subdomain_spaces[ii],
        lambda_hat,
        lambda_u=lambda_xi, lambda_v=lambda_xi_prime,
        kappa=kappa,
        over_integrate=0)
    subdomain_walker.append(diffusive_flux_aa_product)

    # eta df, u x reconstructed_u part
    diffusive_flux_ab_product = make_diffusive_flux_ab_product(
        grid, ii,
        range_space=subdomain_spaces[ii],
        source_space=subdomain_rt_spaces[ii],
        lambda_hat=lambda_hat,
        lambda_range=lambda_xi,
        kappa=kappa,
        over_integrate=0)
    subdomain_walker.append(diffusive_flux_ab_product)

    # eta df, reconstructed_u x reconstructed_u part
    diffusive_flux_bb_product = make_diffusive_flux_bb_product(
        grid, ii,
        subdomain_rt_spaces[ii],
        lambda_hat,
        kappa=kappa,
        over_integrate=0)
    subdomain_walker.append(diffusive_flux_bb_product)

    # walk the grid and assemble/apply
    subdomain_walker.walk()
    penalty_product = penalty_product.matrix()
    elliptic_product = elliptic_product.matrix()
    h_div_semi_product = h_div_semi_product.matrix()
    eta_r_fxf_product = eta_r_fxf_product.result()
    eta_r_fxRu_functional = eta_r_fxRu_functional.vector()
    diffusive_flux_aa_product = diffusive_flux_aa_product.matrix()
    diffusive_flux_ab_product = diffusive_flux_ab_product.matrix()
    diffusive_flux_bb_product = diffusive_flux_bb_product.matrix()

    # localize solution
    subdomain_uh = make_discrete_function(subdomain_spaces[ii], localized_u_h[ii])

    def localize_to_subdomain_with_neighborhood_support(ss):
        return make_discrete_function(
            neighborhood_space,
            neighborhood_space.project_onto_neighborhood(
                [localized_u_h[nn] if nn == ss else Vector(block_space.local_space(nn).size(), 0.)
                 for nn in neighborhood],
                neighborhood))

    subdomain_uh_with_neighborhood_support = {nn: localize_to_subdomain_with_neighborhood_support(nn)
                                              for nn in neighborhood}

    # compute oswald interpolation
    def interpolate_on_neighborhood(nn):
        interpolation_vector = Vector(neighborhood_space.mapper.size, 0.)
        interpolation = make_discrete_function(neighborhood_space, interpolation_vector)
        apply_oswald_interpolation_operator_on_neighborhood(
            grid, ii,
            make_subdomain_boundary_info(grid, {'type': 'xt.grid.boundaryinfo.alldirichlet'}),
            subdomain_uh_with_neighborhood_support[nn],
            interpolation)
        return interpolation_vector

    interpolation_error = {nn: (subdomain_uh_with_neighborhood_support[nn].vector_copy()
                                - interpolate_on_neighborhood(nn))
                           for nn in neighborhood}

    def restrict_from_neighborhood_to_subdomain(neighborhood_vector):
        neighborhood_vector_view = np.array(neighborhood_vector, copy=False)
        subdomain_vector = Vector(subdomain_spaces[ii].size(), 0.)
        subdomain_vector_view = np.array(subdomain_vector, copy=False)
        local_sizes = [subdomain_spaces[ss].size() for ss in neighborhood]
        local_starts = [int(np.sum(local_sizes[:ii])) for ii in range(len(neighborhood))]
        local_starts.append(neighborhood_space.mapper.size)
        index = neighborhood.index(ii)
        subdomain_vector_view[:] = neighborhood_vector_view[local_starts[index]:local_starts[index+1]]
        return subdomain_vector
    
    interpolation_error = {nn: restrict_from_neighborhood_to_subdomain(vec)
                           for nn, vec in interpolation_error.items()}

    # compute flux reconstructions
    def reconstruct_on_neighborhood(nn):
        reconstruction_vector = Vector(neighborhood_rt_spaces[ii].size(), 0.)
        reconstruction = make_discrete_function(neighborhood_rt_spaces[ii], reconstruction_vector)
        apply_diffusive_flux_reconstruction_in_neighborhood(
            grid, ii,
            lambda_xi, kappa,
            subdomain_uh_with_neighborhood_support[nn],
            reconstruction,
            over_integrate=0)
        return subdomain_rt_spaces[ii].restrict(neighborhood_rt_spaces[ii].extend(reconstruction_vector))

    reconstructed_subdomain_uh = {nn: reconstruct_on_neighborhood(nn) for nn in neighborhood}

    # get the results

    # eta r, f x f part
    local_eta_r_squared[ii] += eta_r_fxf_product

    # eta df, u x u part
    local_eta_df_squared[ii] += localized_u_h[ii] * (diffusive_flux_aa_product * localized_u_h[ii])

    for jj in neighborhood:

        # eta r, f x reconstructed_u part
        local_eta_r_squared[ii] -= 2.*(eta_r_fxRu_functional*reconstructed_subdomain_uh[jj])

        # eta df, u x reconstructed_u part
        local_eta_df_squared[ii] += 2.*(localized_u_h[ii]
                                        *(diffusive_flux_ab_product*reconstructed_subdomain_uh[jj]))

        for kk in neighborhood:

            # eta nc
            local_eta_nc_squared[ii] += interpolation_error[jj] * (elliptic_product * interpolation_error[kk])

            # eta r, reconstructed_u x reconstructed_u part
            local_eta_r_squared[ii] += (reconstructed_subdomain_uh[jj]
                                        * (h_div_semi_product * reconstructed_subdomain_uh[kk]))

            # eta df, reconstructed_u x reconstructed_u part
            local_eta_df_squared[ii] += (reconstructed_subdomain_uh[jj]
                                        * (diffusive_flux_bb_product * reconstructed_subdomain_uh[kk]))

            # penalty product
            penalty_products[ii] += (subdomain_uh_with_neighborhood_support[kk].vector_copy()
                                     * (penalty_product * subdomain_uh_with_neighborhood_support[jj].vector_copy()))

    # eta r, scale
    poincaree_constant = 1./(np.pi**2)
    min_diffusion_ev = min_diffusion_eigenvalue(grid, ii, lambda_hat, kappa)
    subdomain_h = subdomain_diameter(grid, ii)
    local_eta_r_squared[ii] *= (poincaree_constant/min_diffusion_ev) * subdomain_h**2

    print('.', end='')

print('')
print('  nonconformity indicator:   {} (should be 1.66e-01)'.format(
    np.sqrt(np.sum(local_eta_nc_squared))))
print('  residual indicator:        {} (should be 1.45e-01)'.format(
    np.sqrt(np.sum(local_eta_r_squared))))
print('  diffusive flux indicator:  {} (should be 3.55e-01)'.format(
    np.sqrt(np.sum(local_eta_df_squared))))

eta_tilde = 0.
eta_tilde +=     np.sqrt(gamma_mu_mu_bar)  * np.sqrt(np.sum(local_eta_nc_squared))
eta_tilde += (1./np.sqrt(alpha_mu_mu_hat)) * np.sqrt(np.sum((  np.sqrt(local_eta_r_squared)
                                                             + np.sqrt(local_eta_df_squared))**2))
eta_tilde *=  1./np.sqrt(alpha_mu_mu_bar)

print('  estimated semi-norm error: {} (should be 0.6565...)'.format(eta_tilde))

print('')
print('  penalty product:                {} (should be 0.2587...)'.format(np.sum(penalty_products)))
eta = np.sqrt(eta_tilde**2 + np.sum(penalty_products))
print('  estimated energy-DG norm error: {} (should be 0.8305...)'.format(eta))